In [14]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [15]:
# Import from the crewAI libray.
from crewai import Agent, Task, Crew

In [16]:
# import library
from dotenv import load_dotenv
import os #needed to call function
import openai

In [17]:
# setup configuration function
def configure():
    load_dotenv()
    
# load configuration
configure()

# get the key
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

Role Playing, Focus and Cooperation

In [18]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	# default is True if not specified
    allow_delegation=False,
	verbose=True
)

In [19]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

# * **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

Tools, Guardrails and Memory

In [22]:
# import crewAi tools
# SerperDevTool, \ #search the web
# ScrapeWebsiteTool, \ #scrape a website
# WebsiteSearchTool #search a website
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool 

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

Instantiate a document scraper tool.
The tool will scrape a page (only 1 URL) of the CrewAI documentation.


- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

In [ ]:
# scrape specific page
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	# tool available to the agent
    tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [25]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  # enable memory
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [27]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide the best support to Andrew Ng from DeepLearningAI by helping him set up a Crew and add memory to it. I should use the tool Read website content to find the guidance needed.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Introduction - CrewAICrewAI home pageEnglishSearch...⌘KAsk AIStart Cloud TrialcrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet StartedIntroductio

In [28]:
# Display the final result as Markdown.
from IPython.display import Markdown
Markdown(result)

Dear Andrew Ng,

Thank you for reaching out with your question about setting up a Crew and adding memory to it. I understand the importance of this task and I am here to provide you with detailed guidance on how to achieve this.

To begin, let's first understand the basics of CrewAI and how it works. CrewAI is a lean, lightning-fast Python framework that empowers developers to create autonomous AI agents tailored to any scenario. CrewAI consists of two main components: Crews and Flows.

Crews are designed to optimize for autonomy and collaborative intelligence, allowing you to create AI teams where each agent has specific roles, tools, and goals. On the other hand, Flows provide structured automations with granular control over workflow execution, ensuring tasks are executed reliably, securely, and efficiently.

Now, let's focus on setting up a Crew and adding memory to it. To add memory to your Crew, you can leverage the role-based agents feature of CrewAI. You can create specialized agents with defined roles, expertise, and goals, such as memory storage and retrieval. These agents can be equipped with custom tools and APIs to interact with external services and data sources, allowing them to store and access memory as needed.

In addition to role-based agents, CrewAI also offers intelligent collaboration capabilities, allowing agents to work together, share insights, and coordinate tasks to achieve complex objectives. Task management in CrewAI allows you to define sequential or parallel workflows, with agents automatically handling task dependencies, including memory-related tasks.

By combining Crews and Flows, you can achieve a balanced approach to autonomy and control in your AI operations. Crews excel at autonomous collaboration, while Flows provide structured workflows for precise task orchestration. This combined approach allows you to leverage the strengths of both components to optimize your AI operations.

I hope this guidance helps you in setting up your Crew and adding memory to it successfully. If you have any further questions or need additional assistance, please feel free to reach out. I am here to support you every step of the way.

Best regards,

[Your Name]
Senior Support Representative
crewAI